<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("/content/2024.csv")

In [4]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

,PlayDate,AP,N01,N02,N03,N04,N05,N06,N07,N08,...,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
674,2023-12-05,A,6,7,8,18,20,21,24,25,...,37,39,44,50,55,56,58,60,61,66
675,2023-12-05,P,3,9,11,13,18,20,23,25,...,44,50,51,52,54,56,58,62,63,66
676,2023-12-06,A,5,6,10,11,14,18,21,22,...,38,46,47,48,49,59,65,67,68,69
677,2023-12-06,P,2,7,11,16,17,20,24,25,...,40,41,43,44,49,52,55,56,60,61
678,2023-12-07,A,7,10,15,17,19,25,28,31,...,45,49,51,56,58,60,61,64,65,67


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
count,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000
mean,3.119293,6.416789,9.793814,13.061856,16.645066,20.248895,23.681885,27.179676,30.618557,33.898380,37.269514,40.522828,43.938144,47.500736,50.824742,53.992636,57.415317,60.712813,64.108984,67.645066
std,2.364729,3.356640,4.114715,4.830368,5.368083,5.766721,5.960946,5.898862,5.998304,5.986956,6.060375,6.128106,5.932686,5.665680,5.490155,5.230521,4.726577,4.357714,3.687245,2.525814
min,1.000000,2.000000,3.000000,4.000000,5.000000,8.000000,11.000000,12.000000,14.000000,17.000000,20.000000,23.000000,25.000000,31.000000,35.000000,37.000000,39.000000,40.000000,46.000000,58.000000
25%,1.000000,4.000000,7.000000,9.000000,13.000000,16.000000,20.000000,23.000000,26.000000,30.000000,33.000000,36.000000,40.000000,44.000000,47.000000,50.500000,54.000000,58.000000,62.000000,66.000000
50%,2.000000,6.000000,9.000000,12.000000,16.000000,20.000000,23.000000,27.000000,31.000000,34.000000,37.000000,40.000000,44.000000,48.000000,51.000000,55.000000,58.000000,61.000000,65.000000,68.000000
75%,4.000000,8.000000,12.000000,16.000000,20.000000,24.000000,27.500000,31.000000,35.000000,37.000000,41.000000,45.000000,48.000000,52.000000,55.000000,58.000000,61.000000,64.000000,67.000000,70.000000
max,15.000000,19.000000,27.000000,28.000000,35.000000,43.000000,46.000000,47.000000,49.000000,51.000000,54.000000,57.000000,58.000000,61.000000,62.000000,64.000000,67.000000,68.000000,69.000000,70.000000


In [9]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [11]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 3
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


679

In [16]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [24]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 30

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [25]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=90)

# Train the model
history = model.fit(X, y, batch_size=100, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

In [27]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

22/22 [==============================] - 3s 11ms/step - loss: 0.0052 - accuracy: 0.8915
Test Loss: 0.005162391811609268, Test Accuracy: 0.891530454158783


In [28]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [87]:
df1 = df.copy()
to_predict = df.tail(7)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-87-4d6bef978192>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
672,7,9,12,14,16,18,20,24,25,30,34,44,45,47,49,51,54,59,64,70
673,1,3,6,7,16,20,31,35,38,42,49,50,51,52,53,55,56,57,63,64
674,6,7,8,18,20,21,24,25,32,36,37,39,44,50,55,56,58,60,61,66
675,3,9,11,13,18,20,23,25,37,41,44,50,51,52,54,56,58,62,63,66
676,5,6,10,11,14,18,21,22,25,29,38,46,47,48,49,59,65,67,68,69
677,2,7,11,16,17,20,24,25,27,30,40,41,43,44,49,52,55,56,60,61


In [90]:
to_predict = np.array(to_predict)
scaled_to_predict = scaler.transform(to_predict)

In [91]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 35ms/step
The predicted numbers in the last lottery game are: [ 7 10 15 17 19 25 28 31 37 39 44 49 51 55 57 60 61 63 64 66]


In [92]:
prediction = df.tail(1)
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])

The actual numbers in the last lottery game were: [ 7 10 15 17 19 25 28 31 37 39 45 49 51 56 58 60 61 64 65 67]


In [93]:
temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("winners:", duplicates)
print(len(duplicates)/20)

winners: [ 7 10 15 17 19 25 28 31 37 39 49 51 60 61 64]
0.75


# Predict the next set of numbers

,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
673,1,3,6,7,16,20,31,35,38,42,49,50,51,52,53,55,56,57,63,64
674,6,7,8,18,20,21,24,25,32,36,37,39,44,50,55,56,58,60,61,66
675,3,9,11,13,18,20,23,25,37,41,44,50,51,52,54,56,58,62,63,66
676,5,6,10,11,14,18,21,22,25,29,38,46,47,48,49,59,65,67,68,69
677,2,7,11,16,17,20,24,25,27,30,40,41,43,44,49,52,55,56,60,61
678,7,10,15,17,19,25,28,31,37,39,45,49,51,56,58,60,61,64,65,67


In [38]:
next = df.copy()
next = next.tail((window_length))
next = np.array(next)

x_next = scaler.transform(next)
next_Date = '2023-12-01 Evening';

y_next_pred = model.predict(np.array([x_next]))
print("The predicted numbers for the lottery game which will take place on",next_Date, "are (without rounding up):\n", scaler.inverse_transform(y_next_pred).astype(int)[0])
print("\n")

temp = np.array(scaler.inverse_transform(y_next_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)
uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("winners:", duplicates)
print(len(duplicates)/20)

1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game which will take place on 2023-12-01 Evening are (without rounding up):
 [ 3  5  7  9 10 17 21 25 28 29 33 36 41 44 47 50 52 57 57 65]


without rounding up
winners: [ 7 10 17 25 28 57 65]
0.35


In [98]:
to_predict_next = df.tail(window_length)
to_predict_next = np.array(to_predict_next)
to_predict_next

array([[ 1,  3,  6,  7, 16, 20, 31, 35, 38, 42, 49, 50, 51, 52, 53, 55,
        56, 57, 63, 64],
       [ 6,  7,  8, 18, 20, 21, 24, 25, 32, 36, 37, 39, 44, 50, 55, 56,
        58, 60, 61, 66],
       [ 3,  9, 11, 13, 18, 20, 23, 25, 37, 41, 44, 50, 51, 52, 54, 56,
        58, 62, 63, 66],
       [ 5,  6, 10, 11, 14, 18, 21, 22, 25, 29, 38, 46, 47, 48, 49, 59,
        65, 67, 68, 69],
       [ 2,  7, 11, 16, 17, 20, 24, 25, 27, 30, 40, 41, 43, 44, 49, 52,
        55, 56, 60, 61],
       [ 7, 10, 15, 17, 19, 25, 28, 31, 37, 39, 45, 49, 51, 56, 58, 60,
        61, 64, 65, 67]])

In [99]:
scaled_to_predict_next = scaler.transform(to_predict_next)
y_pred_next = model.predict(np.array([scaled_to_predict_next]))
print("The predicted numbers for the next draw are:\n", scaler.inverse_transform(y_pred_next).astype(int)[0])

1/1 [==============================] - 0s 21ms/step
The predicted numbers for the next draw are:
 [ 3  5  7  9 10 17 21 25 28 29 33 36 41 44 47 50 52 57 57 65]
